Kornpob Bhirombhakdi, kbhirombhakdi@stsci.edu

# Story
We will study the [materials provided by INS](https://innerspace.stsci.edu/spaces/viewspace.action?key=INSTL) about photometry.

# Previously
We were on the task to perform contamination removal on the field of GRB 190829A at epochs 201909 and 201910. The plan was to investigate if the contaminating object X01 could be considered as a stable star by looking at its photometric property. Therefore, we diverted our effort to develop a pipeline through HSTPHOT package, which should yield better productivity in a long run. However, we encountered several issues as noted in the previous report (20200819). On the same day, INS released training modules of which aperture photometry is one of them. So, we thought that checking the module might be a better idea.

# Resouces
- [INS training library](https://innerspace.stsci.edu/spaces/viewspace.action?key=INSTL)
 - [Photometry](https://innerspace.stsci.edu/display/INSTL/Photometry): it has both slide presentation and self-study Notebook.
 - [Drizzlepac](https://innerspace.stsci.edu/display/INSTL/Using+DrizzlePac)
 - [Basic Spectroscopy](https://innerspace.stsci.edu/display/INSTL/Basic+Spectroscopy)
 - [Synthetic Photometry](https://innerspace.stsci.edu/display/INSTL/Synthetic+Photometry)
 - [Python for Scientific Data Analysis](https://innerspace.stsci.edu/display/INSTL/Python+for+Scientific+Data+Analysis)


# Photometry: PDF Slide Presentation
- Slide 5: it mentions that ACS zeropoints were calibrated with drz products. If using flt, drz = flt * PAM / exptime.
- Slide 5: it notifies complication with computing phot errors: 'DRZ images in cps must be multiplies by EXPTIME and include the background sky.' Note that drz normally removes background sky.
- Slide 6: it explains the different definitions of Vega, ST, and AB magnitude systems.
- Slide 7: it mentions photometric keywords in HST fits header: PHOTMODE, PHOTFLAM, PHOTZP, PHOTPLAM, PHOTBW.
- Slide 7: it mentions formulae to apply the values of photometric keywords (PHOTFLAM, PHOTZP, and PHOTPLAM) to compute ST and AB magnitudes.
- Slide 8: it mentions a formula to calibrate aperture photometry to absolute magnitude in ST system.
- Slide 9: it mentions how aperture correction plays roles in calibration. It also mentions that 'encircled energy curves should be used ... when it is otherwise impossible to determine such corrections directly from the image.' Also, the correction is a function of time and location on the chip.
- Slide 11: it mentions hourly variation of PSF FWHM of HST.
- Slide 14-16 mention CTE correction particularly for ACS/WFC and WFC3/UVIS. Note that WFC3/IR is typically not affected much by this problem.


# Photometry: Notebook Tutorial
- It uses photutils as a main package for performing photometry.
- It does source detection using DAOStarFinder in photutils.
- It does aperture photometry using background estimation from an annulus around each object.
- Cell 15: Convert flux to mag using -2.5*np.log10(flux).
- Cell 16 - 18: Compute aperture correction by
 - Selecting only good stars by using bounding box + sigma clipping.
 - Correcting magnitude measured with aperture radius 3 to be radius 5 by using the aperture correction.
 - Applying zeropoint.
- Exercise requires images produced from [Drizzlepac training module](https://innerspace.stsci.edu/pages/viewpage.action?spaceKey=INSTL&title=Using+DrizzlePac).
 - One key point is to set mdriztab = False when specifying your own parameters during optimization.


# Any idea how to fix HSTPHOT?
- should aperture correction be applied to flt or drz? The training modules do not explicitly mention anything regarding this issue. However, from the review of WFC3 Instrument Handbook, it seems like drz is the right one.
- how to properly compute emag? From the Photometry training module, aperture_photometry is done on the original non-subtracted image. This will provide mag and emag. Then, background contribution is subtracted separately to yield the final mag result. However, it does not add errror from the estimated background subtraction, which is understandable if one assumes negligible contribution or the subtraction being deterministic. This sounds correct according to how aperture_photometry works. Therefore, ApPhot should take both original image and background image. Then, perform aperture_photometry on the original image with correct effective gain and without background error contribution. Then, if background image is supplied, perform aperture_photometry on the background image given the same aperture radius. Output both components separately as well as subtraction. Error will come only from the original computed by the calc_total_error as the poisson source, and background contribution will provide in the calc_total_error with the right keyword.


# Next
- Let's start with developing a routine to do pysynphot. Then, apply to GD153 given bandpass that we will test the HSTPHOT with, so we have a better idea of what should be the correct result.
- Then, we fix HSTPHOT, and test it.
